In [1]:
# importing tools 
import s3fs
import xarray as xr
import numpy as np 
import matplotlib.pyplot as plt 
#for removing warnings
import warnings

In [2]:
# open the data with s3fs from the databucket 
fsg = s3fs.S3FileSystem(anon=False,
      client_kwargs={
         'endpoint_url': 'https://karen.uiogeo-apps.sigma2.no'
      })
data_path1 = 's3://data/CREG12.L75-REF08_mesh_zgr.zarr'
data_path2 = 's3://velocity-u.zarr'
data_path3 = 's3://velocity-v.zarr'

remote_files1 = fsg.glob(data_path1)
remote_files2 = fsg.glob(data_path2)
remote_files3 = fsg.glob(data_path3)

store1 = s3fs.S3Map(root=data_path1, s3=fsg, check=False)
store2 = s3fs.S3Map(root=data_path2, s3=fsg, check=False)
store3 = s3fs.S3Map(root=data_path3, s3=fsg, check=False)

dzz = xr.open_zarr(store=store1, consolidated=True)
du = xr.open_zarr(store=store2, consolidated=True)
dv = xr.open_zarr(store=store3, consolidated=True)

In [3]:
def open_s3fs(path):
    # open the data with s3fs from the databucket 
    fsg = s3fs.S3FileSystem(anon=False,
      client_kwargs={
         'endpoint_url': 'https://karen.uiogeo-apps.sigma2.no'
      })
    data_path = f's3://data/{path}'
    remote_files = fsg.glob(data_path)
    fileset = [fsg.open(file) for file in remote_files]
    #open the dataset 
    dset = xr.open_mfdataset(fileset, 
                             #combine='by_coords',
                             compat='override')
    return dset

In [4]:
scalar = open_s3fs('CREG12.L75-REF08_mesh_hgr.nc')

In [5]:
p_list = [(31.13247,81.24202),
          (31.13533,81.24255),
          (31.14506,81.24587)
          ,(011.1189, 69.5289),
          (013.16845,68.58759),
          (013.19866,68.56109),
          (012.45082,68.50128),
          (5.57541,79.37209),
          (5.48733,80.03876),
          (5.56333,79.44093),
          (-160.4923,72.121),
          (-159.1216,72.1628),
          (-158.5512,72.1815),
          (-163.5346,72.2808),
          (-164.0992,72.5252)]
moor_l = [(836, 440),
            (838, 440),
            (836, 439),
            (915, 157),
            (935, 143),
            (938, 144),
            (934, 141),
            (774, 344),
            (766, 358),
            (773, 346),
            (334, 978),
            (334, 978),
            (334, 978),
            (334, 978),
            (409, 992), 
            (399, 987), 
            (395, 985), 
            (434, 997), 
            (440, 993)]
c_list = ['c', 'orangered', 'g',
          'r', 'm', 'y','tab:orange',
          'tab:pink', 'limegreen', 'maroon', 
          'dodgerblue', 'gold', 'peru','deeppink',
          'gold','peru','dodgerblue','c','m']

In [49]:
u = du.vozocrtx.isel(x=slice(200,1400),y=slice(650,1800))
u = u.rename({'depthu' : 'depth'})
v = dv.vomecrty.isel(x=slice(200,1400),y=slice(650,1800))
v = v.rename({'depthv' : 'depth'})

bathym = dzz.mbathy.isel(x=slice(200,1400),y=slice(650,1800)).squeeze(axis=0)

zonal = scalar.glamt.isel(x=slice(200,1399),y=slice(650,1799)).squeeze(axis=0)
merd = scalar.gphit.isel(x=slice(200,1399),y=slice(650,1799)).squeeze(axis=0)

In [28]:
d_hbx = bathym.isel(y=slice(0,1149)).diff(dim='x')
d_hby = bathym.isel(x=slice(0,1199)).diff(dim='y')

len_bat = np.sqrt((d_hbx/zonal)**2 + (d_hby/merd)**2)

d_hbxg = d_hbx.rolling(x = 10,).mean()
d_hbyg = d_hby.rolling(y = 10,).mean()

In [43]:
def PCz_20(x,y):
    'Finding the level the given location dosent have values for anymore'

    d = bathym.isel(x=x, y=y).values

    dypu = u.isel(x=x, y=y, depth = slice(15,d-1)) # -1 pga d=0 og vi vil en opp der vi har bunnhastigheter
    dypv = v.isel(x=x, y=y, depth = slice(15,d-1))
    
    covu = np.cov(dypu.T, bias=True) # uten .T ble den en 73x73 matrise 
    values_u, vectors_u = np.linalg.eig(covu)
    order_u = values_u.argsort()[::-1]
    values_u, vectors_u = values_u[order_u], vectors_u[:, order_u]
    
    covv = np.cov(dypv.T, bias=True) # uten .T ble den en 73x73 matrise 
    values_v, vectors_v = np.linalg.eig(covv)
    order_v = values_v.argsort()[::-1]
    values_v, vectors_v = values_v[order_v], vectors_v[:, order_v]
    
    PC1u = (np.sqrt(values_u[0])*vectors_u[:,0])
    PC1v = (np.sqrt(values_v[0])*vectors_v[:,0])
    PC2u = (np.sqrt(values_u[1])*vectors_u[:,1])
    PC2v = (np.sqrt(values_v[1])*vectors_v[:,1])
    
    return  PC1u, PC1v, PC2u, PC2v, dypu, dypv

In [44]:
def PCz(x,y):
    'Finding the level the given location dosent have values for anymore'

    d = bathym.isel(x=x, y=y).values

    dypu = u.isel(x=x, y=y, depth = slice(0,d-1)) # -1 pga d=0 og vi vil en opp der vi har bunnhastigheter
    dypv = v.isel(x=x, y=y, depth = slice(0,d-1))
    
    covu = np.cov(dypu.T, bias=True) # uten .T ble den en 73x73 matrise 
    values_u, vectors_u = np.linalg.eig(covu)
    order_u = values_u.argsort()[::-1]
    values_u, vectors_u = values_u[order_u], vectors_u[:, order_u]
    
    covv = np.cov(dypv.T, bias=True) # uten .T ble den en 73x73 matrise 
    values_v, vectors_v = np.linalg.eig(covv)
    order_v = values_v.argsort()[::-1]
    values_v, vectors_v = values_v[order_v], vectors_v[:, order_v]
    
    PC1u = (np.sqrt(values_u[0])*vectors_u[:,0])
    PC1v = (np.sqrt(values_v[0])*vectors_v[:,0])
    PC2u = (np.sqrt(values_u[1])*vectors_u[:,1])
    PC2v = (np.sqrt(values_v[1])*vectors_v[:,1])
    
    return  PC1u, PC1v, PC2u, PC2v, dypu, dypv

In [34]:
'Making a function for the vertical PC'
def PCz_op(x,y):
    up = u.isel(y=y, x=x)
    vp = v.isel(y=y, x=x)

    zonalp = zonal.isel(y=y, x=x)
    merdp = merd.isel(y=y, x=x)

    d_hbx_p = d_hbxg.isel(y=y, x=x)
    d_hby_p = d_hbyg.isel(y=y, x=x)
    
    # lag lengde scalar av bathym og dele u_o og u_p på 

    vel_ort = ((up*(d_hbx_p/zonalp) + vp*(d_hby_p/merdp))/np.sqrt((d_hbx_p/zonalp)**2 + (d_hby_p/merdp)**2)) 
    vel_par = ((up*(d_hby_p/merdp)  - vp*(d_hbx_p/zonalp))/np.sqrt((d_hbx_p/zonalp)**2 + (d_hby_p/merdp)**2))
    
    d_p = bathym.isel(y=y, x=x).values
    dyp_o = vel_ort.isel(depth = slice(0,int(d_p)-1)) # -1 pga d=0 og vi vil en opp der vi har bunnhastigheter
    dyp_p = vel_par.isel(depth = slice(0,int(d_p)-1))

    cov_o = np.cov(dyp_o.T, bias=True) # uten .T ble den en 73x73 matrise 
    cov_p = np.cov(dyp_p.T, bias=True)
    
    values_o, vectors_o = np.linalg.eig(cov_o)
    order_o = values_o.argsort()[::-1]
    values_o, vectors_o = values_o[order_o], vectors_o[:, order_o]
    values_p, vectors_p = np.linalg.eig(cov_p)
    order_p = values_p.argsort()[::-1]
    values_p, vectors_p = values_p[order_p], vectors_p[:, order_p]
    
    PC1p = (np.sqrt(values_p[0])*vectors_p[:,0])
    PC1o = (np.sqrt(values_o[0])*vectors_o[:,0])
    PC2p = (np.sqrt(values_p[1])*vectors_p[:,1])
    PC2o = (np.sqrt(values_o[1])*vectors_o[:,1])
    
    return PC1p, PC1o, PC2p, PC2o, dyp_p, dyp_o

## Barents Sea

In [ ]:
warnings.simplefilter("ignore")
fig, axes = plt.subplots(1,3,figsize = (30,13))
plt.suptitle('PC1 & PC2 from model data, Barents Sea', 
             horizontalalignment='center',
            fontsize=24)
bar = [(836, 440),(838, 440),(836, 439)]
axes[0].set_ylabel('Depth [m]', fontsize=14)

for axs, p in zip(axes.flat, range(len(bar))):
    pp = moor_l[p]
    print(pp[0], pp[1])    
    PC1u, PC1v, PC2u, PC2v, dypu, dypv = PCz(pp[0],pp[1])
        
    axs.plot(PC1u,dypu.depth, 'b', label = 'U-Velocity, PC1')
    axs.plot(PC1v,dypv.depth, 'deepskyblue', label = 'V-Velocity, PC1')
    axs.plot(PC2u,dypu.depth, 'b', linestyle = '--', label = 'U-Velocity, PC2')
    axs.plot(PC2v,dypv.depth, 'deepskyblue', linestyle = '--', label = 'V-Velocity, PC2')
    axs.axvline(0, color='k', linestyle = '--')
    axs.legend(frameon=False, fontsize=12, loc="upper left") 
    axs.invert_yaxis()
    axs.set_xlabel('Current speed [cm/s]', fontsize=14)


836 440


In [ ]:
warnings.simplefilter("ignore")
fig, axes = plt.subplots(1,3,figsize = (30,13))
plt.suptitle('PC1 from model data, Barents Sea, oriented parallel & orthogonal to the topography', 
             horizontalalignment='center',
            fontsize=24)
bar = [(836, 440),(838, 440),(836, 439)]
axes[0].set_ylabel('Depth [m]', fontsize=14)

for axs, p in zip(axes.flat, range(len(bar))):
    pp = moor_l[p]
    print(pp[0], pp[1])    
    PC1p, PC2o, PC2p, PC2o, dypp, dypo = PCz(pp[0],pp[1])
        
    axs.plot(PC1p,dypp.depth, 'navy', label = 'Velocity parallel')
    axs.plot(PC1o,dypo.depth, 'cornflowerblue', label = 'Velocity orthogonal')
    axs.plot(PC2p,dypp.depth, 'navy', linestyle = '--', label = 'Velocity parallel')
    axs.plot(PC2o,dypo.depth, 'cornflowerblue', linestyle = '--', label = 'Velocity orthogonal')
    axs.axvline(0, color='k', linestyle = '--')
    axs.legend(frameon=False, fontsize=12, loc="upper left") 
    axs.invert_yaxis()
    axs.set_xlabel('Current speed [cm/s]', fontsize=14)

## Lofoten Basin 

In [ ]:
lof = [(915, 157),(935, 143),(938, 144),(934, 141)]

In [ ]:
warnings.simplefilter("ignore")
fig, axes = plt.subplots(1,3,figsize = (30,13))
plt.suptitle('PC1 & PC2 from model data, Lofoten Basin', 
             horizontalalignment='center',
            fontsize=24)
axes[0].set_ylabel('Depth [m]', fontsize=14)

for axs, p in zip(axes.flat, range(len(lof))):
    pp = moor_l[p]
    print(pp[0], pp[1])    
    PC1u, PC1v, PC2u, PC2v, dypu, dypv = PCz(pp[0],pp[1])
        
    axs.plot(PC1u,dypu.depth, 'b', label = 'U-Velocity, PC1')
    axs.plot(PC1v,dypv.depth, 'deepskyblue', label = 'V-Velocity, PC1')
    axs.plot(PC2u,dypu.depth, 'b', linestyle = '--', label = 'U-Velocity, PC2')
    axs.plot(PC2v,dypv.depth, 'deepskyblue', linestyle = '--', label = 'V-Velocity, PC2')
    axs.axvline(0, color='k', linestyle = '--')
    axs.legend(frameon=False, fontsize=12, loc="upper left") 
    axs.invert_yaxis()
    axs.set_xlabel('Current speed [cm/s]', fontsize=14)


In [ ]:
warnings.simplefilter("ignore")
fig, axes = plt.subplots(1,3,figsize = (30,13))
plt.suptitle('PC1 from model data, Lofoten Basin, oriented parallel & orthogonal to the topography', 
             horizontalalignment='center',
            fontsize=24)
bar = [(836, 440),(838, 440),(836, 439)]
axes[0].set_ylabel('Depth [m]', fontsize=14)

for axs, p in zip(axes.flat, range(len(bar))):
    pp = moor_l[p]
    print(pp[0], pp[1])    
    PC1p, PC2o, PC2p, PC2o, dypp, dypo = PCz(pp[0],pp[1])
        
    axs.plot(PC1p,dypp.depth, 'navy', label = 'Velocity parallel')
    axs.plot(PC1o,dypo.depth, 'cornflowerblue', label = 'Velocity orthogonal')
    axs.plot(PC2p,dypp.depth, 'navy', linestyle = '--', label = 'Velocity parallel')
    axs.plot(PC2o,dypo.depth, 'cornflowerblue', linestyle = '--', label = 'Velocity orthogonal')
    axs.axvline(0, color='k', linestyle = '--')
    axs.legend(frameon=False, fontsize=12, loc="upper left") 
    axs.invert_yaxis()
    axs.set_xlabel('Current speed [cm/s]', fontsize=14)

## Yermark shelf

In [ ]:
yer = [(774, 344),(766, 358),(773, 346)]

In [ ]:
warnings.simplefilter("ignore")
fig, axes = plt.subplots(1,3,figsize = (30,13))
plt.suptitle('PC1 & PC2 from model data, Yermark Shelf', 
             horizontalalignment='center',
            fontsize=24)
axes[0].set_ylabel('Depth [m]', fontsize=14)

for axs, p in zip(axes.flat, range(len(yer))):
    pp = moor_l[p]
    print(pp[0], pp[1])    
    PC1u, PC1v, PC2u, PC2v, dypu, dypv = PCz(pp[0],pp[1])
        
    axs.plot(PC1u,dypu.depth, 'b', label = 'U-Velocity, PC1')
    axs.plot(PC1v,dypv.depth, 'deepskyblue', label = 'V-Velocity, PC1')
    axs.plot(PC2u,dypu.depth, 'b', linestyle = '--', label = 'U-Velocity, PC2')
    axs.plot(PC2v,dypv.depth, 'deepskyblue', linestyle = '--', label = 'V-Velocity, PC2')
    axs.axvline(0, color='k', linestyle = '--')
    axs.legend(frameon=False, fontsize=12, loc="upper left") 
    axs.invert_yaxis()
    axs.set_xlabel('Current speed [cm/s]', fontsize=14)


In [ ]:
warnings.simplefilter("ignore")
fig, axes = plt.subplots(1,3,figsize = (30,13))
plt.suptitle('PC1 from model data, Yermark Shelf, oriented parallel & orthogonal to the topography', 
             horizontalalignment='center',
            fontsize=24)
axes[0].set_ylabel('Depth [m]', fontsize=14)

for axs, p in zip(axes.flat, range(len(yer))):
    pp = moor_l[p]
    print(pp[0], pp[1])    
    PC1p, PC2o, PC2p, PC2o, dypp, dypo = PCz(pp[0],pp[1])
        
    axs.plot(PC1p,dypp.depth, 'navy', label = 'Velocity parallel')
    axs.plot(PC1o,dypo.depth, 'cornflowerblue', label = 'Velocity orthogonal')
    axs.plot(PC2p,dypp.depth, 'navy', linestyle = '--', label = 'Velocity parallel')
    axs.plot(PC2o,dypo.depth, 'cornflowerblue', linestyle = '--', label = 'Velocity orthogonal')
    axs.axvline(0, color='k', linestyle = '--')
    axs.legend(frameon=False, fontsize=12, loc="upper left") 
    axs.invert_yaxis()
    axs.set_xlabel('Current speed [cm/s]', fontsize=14)

## Beaufort Sea

In [ ]:
bea = [(334, 978), (334, 978), (334, 978), (334, 978)]

In [ ]:
warnings.simplefilter("ignore")
fig, axes = plt.subplots(1,3,figsize = (30,13))
plt.suptitle('PC1 & PC2 from model data, Beaufort Sea', 
             horizontalalignment='center',
            fontsize=24)
axes[0].set_ylabel('Depth [m]', fontsize=14)

for axs, p in zip(axes.flat, range(len(bea))):
    pp = moor_l[p]
    print(pp[0], pp[1])    
    PC1u, PC1v, PC2u, PC2v, dypu, dypv = PCz(pp[0],pp[1])
        
    axs.plot(PC1u,dypu.depth, 'b', label = 'U-Velocity, PC1')
    axs.plot(PC1v,dypv.depth, 'deepskyblue', label = 'V-Velocity, PC1')
    axs.plot(PC2u,dypu.depth, 'b', linestyle = '--', label = 'U-Velocity, PC2')
    axs.plot(PC2v,dypv.depth, 'deepskyblue', linestyle = '--', label = 'V-Velocity, PC2')
    axs.axvline(0, color='k', linestyle = '--')
    axs.legend(frameon=False, fontsize=12, loc="upper left") 
    axs.invert_yaxis()
    axs.set_xlabel('Current speed [cm/s]', fontsize=14)


In [ ]:
warnings.simplefilter("ignore")
fig, axes = plt.subplots(1,3,figsize = (30,13))
plt.suptitle('PC1 from model data, Beaufort Sea, oriented parallel & orthogonal to the topography', 
             horizontalalignment='center',
            fontsize=24)
axes[0].set_ylabel('Depth [m]', fontsize=14)

for axs, p in zip(axes.flat, range(len(bea))):
    pp = moor_l[p]
    print(pp[0], pp[1])    
    PC1p, PC2o, PC2p, PC2o, dypp, dypo = PCz(pp[0],pp[1])
        
    axs.plot(PC1p,dypp.depth, 'navy', label = 'Velocity parallel')
    axs.plot(PC1o,dypo.depth, 'cornflowerblue', label = 'Velocity orthogonal')
    axs.plot(PC2p,dypp.depth, 'navy', linestyle = '--', label = 'Velocity parallel')
    axs.plot(PC2o,dypo.depth, 'cornflowerblue', linestyle = '--', label = 'Velocity orthogonal')
    axs.axvline(0, color='k', linestyle = '--')
    axs.legend(frameon=False, fontsize=12, loc="upper left") 
    axs.invert_yaxis()
    axs.set_xlabel('Current speed [cm/s]', fontsize=14)

## Chuckchi Sea

In [ ]:
chi = [(409, 992), (399, 987), (395, 985), (434, 997), (440, 993)]

In [ ]:
warnings.simplefilter("ignore")
fig, axes = plt.subplots(1,3,figsize = (30,13))
plt.suptitle('PC1 & PC2 from model data, Chuckchi Sea', 
             horizontalalignment='center',
            fontsize=24)
axes[0].set_ylabel('Depth [m]', fontsize=14)

for axs, p in zip(axes.flat, range(len(chi))):
    pp = moor_l[p]
    print(pp[0], pp[1])    
    PC1u, PC1v, PC2u, PC2v, dypu, dypv = PCz(pp[0],pp[1])
        
    axs.plot(PC1u,dypu.depth, 'b', label = 'U-Velocity, PC1')
    axs.plot(PC1v,dypv.depth, 'deepskyblue', label = 'V-Velocity, PC1')
    axs.plot(PC2u,dypu.depth, 'b', linestyle = '--', label = 'U-Velocity, PC2')
    axs.plot(PC2v,dypv.depth, 'deepskyblue', linestyle = '--', label = 'V-Velocity, PC2')
    axs.axvline(0, color='k', linestyle = '--')
    axs.legend(frameon=False, fontsize=12, loc="upper left") 
    axs.invert_yaxis()
    axs.set_xlabel('Current speed [cm/s]', fontsize=14)


In [ ]:
warnings.simplefilter("ignore")
fig, axes = plt.subplots(1,3,figsize = (30,13))
plt.suptitle('PC1 from model data, Chukchi Sea, oriented parallel & orthogonal to the topography', 
             horizontalalignment='center',
            fontsize=24)
axes[0].set_ylabel('Depth [m]', fontsize=14)

for axs, p in zip(axes.flat, range(len(chi))):
    pp = moor_l[p]
    print(pp[0], pp[1])    
    PC1p, PC2o, PC2p, PC2o, dypp, dypo = PCz(pp[0],pp[1])
        
    axs.plot(PC1p,dypp.depth, 'navy', label = 'Velocity parallel')
    axs.plot(PC1o,dypo.depth, 'cornflowerblue', label = 'Velocity orthogonal')
    axs.plot(PC2p,dypp.depth, 'navy', linestyle = '--', label = 'Velocity parallel')
    axs.plot(PC2o,dypo.depth, 'cornflowerblue', linestyle = '--', label = 'Velocity orthogonal')
    axs.axvline(0, color='k', linestyle = '--')
    axs.legend(frameon=False, fontsize=12, loc="upper left") 
    axs.invert_yaxis()
    axs.set_xlabel('Current speed [cm/s]', fontsize=14)